# CZX vertical canonical form

In [50]:
using TensorOperations, LinearAlgebra, PrettyTables

In [4]:
# one-site MPDO

shape = (3, 3, 2, 2)    # alpha, beta, i, j
M = zeros(Float64, shape...)  
M11 = 0.5*[1 0 0; 0 0 0; 0 0 0];
M22 = 0.5*[1 0 0; 0 0 0; 0 0 0];
M12 = 0.5*[0 0 0; 0 1 1; 0 0 0];
M21 = 0.5*[0 0 0; 0 0 0; 0 1 -1];

M[:,:,1,1] = M11;
M[:,:,2,2] = M22;
M[:,:,1,2] = M12;
M[:,:,2,1] = M21;

In [3]:
function block_M(M1, M2)
    D, _ , d1, _ = size(M1)
    _, _ , d2, _ = size(M2)
    Mc = zeros(Float64, D, D, d1*d2, d1*d2)

    @tensor Mc[alpha, beta, i1,i2,j1,j2] := M1[alpha,gamma,i1,j1] * M2[gamma,beta,i2,j2] 
    M_perm = permutedims(Mc, (1, 2, 4, 3, 6, 5))   # to make the result row-major
    Mc_reshape = reshape(M_perm, (D, D, d1*d2, d1*d2))

    return Mc_reshape
end


function MPDO_to_MPV_h(M)
    # combine indices i and j
    D, _ , d, _ = size(M)
    M_perm = permutedims(M, (1, 2, 4, 3)) 
    M_reshape = reshape(M_perm, (D, D, d*d))

    return M_reshape
end


function MPDO_to_MPV_v(M)
    # combine indices alpha and beta
    D, _ , d, _ = size(M)
    M_perm = permutedims(M, (2, 1, 3, 4)) 
    M_reshape = reshape(M_perm, (D*D, d, d))
    M_reshape_perm = permutedims(M_reshape, (2,3,1))

    return M_reshape_perm
end


function transfer_matr(M1, M2)
    D, _ , d = size(M1)
    @tensor T[alphap, alpha,betap, beta] := M1[alpha, beta, i] * conj(M2[alphap, betap, i])
    T_perm = permutedims(T, (2, 1, 4, 3))
    T_reshape = reshape(T_perm, D*D, D*D)
    
    return T_reshape
end

function examine_BNT(M_set)
    # examine the BNT elements are independent
    M_set_MPV = MPDO_to_MPV_v.(M_set);

    n = length(M_set)
    overlap = zeros(ComplexF64, n, n)
    overlap2 = zeros(ComplexF64, n, n)
    for i in 1:n
        for j in 1:n
            T_mix = transfer_matr(M_set_MPV[i], M_set_MPV[j])
            eig_T = eigen(T_mix)
            overlap[i,j] = eig_T.values[end]
            overlap2[i,j] = eig_T.values[end-1]
        end
    end

    return overlap, overlap2
end


function dependent_block(M1, M2)
    M1_MPV = MPDO_to_MPV_v(M1)
    M2_MPV = MPDO_to_MPV_v(M2)

    T_mix = transfer_matr(M1_MPV, M2_MPV)
    T_two = transfer_matr(M2_MPV, M2_MPV)
    eig_Tm = eigen(T_mix)
    eig_T2 = eigen(T_two)

    leading_vec_m = eig_Tm.vectors[:,end]
    leading_vec_2 = eig_T2.vectors[:,end]

    return leading_vec_m, leading_vec_2

end


function examine_single_BNT(M)
    T = transfer_matr(M, M)
    eig_T = eigen(T)
    eig_Tt = eigen(T')

    eval = eig_T.values
    leading_vec = eig_T.vectors[:, end]
    leading_vec_t = eig_Tt.vectors[:, end]

    return eval, leading_vec, leading_vec_t
end

examine_single_BNT (generic function with 1 method)

### Blocking two sites of M

In [5]:
M2 = block_M(M, M);
U = [1 0 0 0; 0 0 0 1; 0 1 0 0; 0 0 1 0]*1.0;
Ut = transpose(U);
 # this is in vertical canonical form
@tensor M2_U[alpha,beta,i,j] := U[i,ip] * M2[alpha,beta,ip,jp] * Ut[jp,j] * 4/sqrt(3);

In [42]:
M2[2,3,:,:]

4×4 Matrix{Float64}:
 0.0  0.0   0.0   0.25
 0.0  0.0  -0.25  0.0
 0.0  0.0   0.0   0.0
 0.0  0.0   0.0   0.0

In [6]:
P1 = [1 0 0 0; 0 1 0 0];
P2 = [0 0 1 0; 0 0 0 1];

@tensor K_blk_one[alpha,beta,i,j] := P1[i,ip] * M2_U[alpha,beta,ip,jp] * P1[j,jp];
@tensor K_blk_two[alpha,beta,i,j] := P2[i,ip] * M2_U[alpha,beta,ip,jp] * P2[j,jp];

K_set = [K_blk_one, K_blk_two]
overlap, overlap2 = examine_BNT(K_set);

In [7]:
overlap

2×2 Matrix{ComplexF64}:
      1.0+0.0im          0.333333+2.94613e-18im
 0.333333+2.94613e-18im       1.0+0.0im

In [8]:
M2_MPV = MPDO_to_MPV_v(M2_U)
T = transfer_matr(M2_MPV, M2_MPV)
eig_T = eigen(T)
eig_T.values

16-element Vector{ComplexF64}:
 -0.3333333333333334 + 0.0im
 -0.3333333333333332 + 0.0im
  0.3333333333333333 + 0.0im
  0.3333333333333334 - 2.9461301385306534e-18im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 0.0im
  0.3333333333333334 + 2.9461301385306534e-18im
                 1.0 + 0.0im
  1.0000000000000002 + 0.0im

### Block four sites of M

In [9]:
M4 = block_M(M2, M2);

In [10]:
U0 = zeros(Float64, 16, 16) 
for i=1:8
    U0[2*i-1,i] = 1;
    U0[2*i, 17-i] = 1;
end

Z = [1 0; 0 -1]
Uz = diagm(0 => ones(16))
Uz[7:8, 7:8] = Z
Uz[13:14, 13:14] = Z

p = [1,2,5,6,9,10,13,14,3,4,7,8,11,12,15,16]
Perm = Matrix{Float64}(I, 16, 16)[p, :]

V = Perm * Uz * U0

@tensor M4_U[alpha,beta,i,j] := V[i,ip] * M4[alpha,beta,ip,jp] * V[j,jp] * 16/sqrt(3);

In [11]:
M4_blk_1 = M4_U[:,:,1:2,1:2];
M4_blk_2 = M4_U[:,:,9:10,9:10];

M4_blk_set = [M4_blk_1, M4_blk_2];  # BNT

e_val, leading_vec, leading_vec_t = examine_single_BNT(MPDO_to_MPV_v(M4_blk_1));
overlap, overlap2 = examine_BNT(M4_blk_set);

In [12]:
abs.(overlap)

2×2 Matrix{Float64}:
 1.0       0.333333
 0.333333  1.0

In [13]:
# examine the canonical form
nb = [1,1,1,1,2,2,2,2]
for i in 1:8
    println(norm(M4_U[:,:,2*i-1:2*i,2*i-1:2*i] - M4_blk_set[nb[i]]))
end


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [13]:
# M4_blk_3 = M4_U[:,:,7:8,7:8];
# M4_blk_4 = M4_U[:,:,13:14,13:14];
# leading_vec_m, leading_vec_2 = dependent_block(M4_blk_2, M4_blk_3);
# Z = [1 0; 0 -1]
# Z * M4_blk_2[3,3,:,:] * Z
# M4_blk_3[3,3,:,:]

## RG channel

In [14]:
m1 = sqrt(3)/4
m2 = sqrt(3)/4

nu1 = diagm(0 => ones(4))*sqrt(3)/16
nu2 = diagm(0 => ones(4))*sqrt(3)/16;

In [15]:
function random_density_matrix(n::Int)
    A = randn(ComplexF64, n, n)  # complex Ginibre matrix
    ρ = A * A'                   # Hermitian and positive semidefinite
    ρ ./= tr(ρ)                  # normalize to trace 1
    return ρ
end


function is_valid_dm(rho::Array{ComplexF64})

    eval = eigen(rho).values

    println("Hermitian? ", ishermitian(rho))
    println("Trace=1? ", isapprox(tr(rho), 1))
    println("Positive? ", (eval).>0)

    return eval
end


function channel_T(rho)
    P1 = zeros(Float64, 2,4)
    P2 = zeros(Float64, 2,4)
    P1[1:2,1:2] = diagm(0 => ones(2))
    P2[1:2,3:4] = diagm(0 => ones(2))

    blk1 = kron(nu1/m1, P1*U*rho*U'*P1')
    blk2 = kron(nu2/m2, P2*U*rho*U'*P2')
    zmtr = zeros(Float64, 8,8)
    dsum = [blk1 zmtr; zmtr blk2]
    rho_out = V'*dsum*V

    return rho_out
end


function partial_trace(ρ::AbstractMatrix, dA::Int, dB::Int)

    # trace over A
    ρ_tensor = permutedims(reshape(ρ', dB, dA, dB, dA), (4,3,2,1))  # make everything row-major

    pt = zeros(eltype(ρ), dB, dB)
    for i in 1:dA
        pt = pt + ρ_tensor[i,:,i,:]
    end

    return pt
end


function channel_S(rho)
    d = 8
    P1 = zeros(Float64, d, 2*d)
    P2 = zeros(Float64, d, 2*d)
    P1[1:d,1:d] = diagm(0 => ones(d))
    P2[1:d,d+1:2*d] = diagm(0 => ones(d))

    blk1 = partial_trace(P1*V*rho*V'*P1',4,2)
    blk2 = partial_trace(P2*V*rho*V'*P2',4,2)

    zmtr = zeros(Float64, 2,2)
    dsum = [blk1 zmtr; zmtr blk2]
    rho_out = U'*dsum*U

end

channel_S (generic function with 1 method)

In [16]:
rand_rho = random_density_matrix(4)
is_valid_dm(channel_T(rand_rho))

rand_rho = random_density_matrix(16)
is_valid_dm(channel_S(rand_rho));

Hermitian? true
Trace=1? true
Positive? Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Hermitian? true
Trace=1? true
Positive? Bool[1, 1, 1, 1]


In [17]:
for alpha in 1:3
    for beta in 1:3
        println(norm(channel_T(M2[alpha,beta,:,:])-M4[alpha,beta,:,:]))
    end
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [18]:
for alpha in 1:3
    for beta in 1:3
        println(norm(channel_S(M4[alpha,beta,:,:])-M2[alpha,beta,:,:]))
    end
end

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [92]:
# test = reshape(1:16, 4, 4)'
# testT = channel_T(test)
# testT[:,9:16]

### Lindbladian

In [19]:
function channel_E(rho)
    return channel_T(channel_S(rho))
end


function func_to_map(f::Function,d::Int)
    
    func_map = zeros(ComplexF64, d^2, d^2)
    input_vecs = diagm(0 => ones(d^2))

    for i=1:d^2
        out_matr = channel_E(reshape(input_vecs[i,:], (d,d))')
        func_map[:,i] = reshape(out_matr', d^2)
    end

    return func_map
end

func_to_map (generic function with 1 method)

In [ ]:
d = 16
matr_channel_E = func_to_map(channel_E, d)
eig_E = eigen(matr_channel_E)
eig_E.values

256-element Vector{Float64}:
 -1.7494466769025583e-16
 -1.3491402892271498e-16
 -1.0492465463812014e-16
 -9.800224918278825e-17
 -9.619849424162404e-17
 -7.396900487809013e-17
 -7.35056112421738e-17
 -7.021666937153401e-17
 -5.246232731906007e-17
 -4.664488705472935e-17
  ⋮
  8.881784197001252e-16
  0.9999999999999982
  0.9999999999999984
  1.0
  1.0000000000000002
  1.0000000000000002
  1.0000000000000002
  1.0000000000000002
  1.0000000000000002

In [62]:
# pretty_table(real(reshape(eig_E.vectors[:,end], (d,d))'));

In [43]:
function apply_channel(rho,nA::Int,nB::Int;act='A')

    dA = 2^nA
    dB = 2^nB
    d = size(rho)[1]

    @assert dA*dB == d
    rho_rs = permutedims(reshape(rho', dB, dA, dB, dA), (4,3,2,1))  # make everything row-major

    out_rho = zeros(eltype(rho), (dA,dB,dA,dB))
    if act == 'A'
        for i in 1:dB
            for j in 1:dB
                out_rho[:,i,:,j] = channel_E(rho_rs[:,i,:,j])
            end
        end
    elseif act == 'B'
        for i in 1:dA
            for j in 1:dA
                out_rho[i,:,j,:] = channel_E(rho_rs[i,:,j,:])
            end
        end
    end

    out_rho_rs = reshape(permutedims(out_rho, (4,3,2,1)), (d,d))'

    return out_rho_rs
end

apply_channel (generic function with 1 method)

In [48]:
# site = 5  # [E_i, E_{i+1}] =0 
# site = 6   # [E_i, E_{i+2}] =0 
# site = 7    # [E_i, E_{i+3}] =0 

let
    for site in 5:7
        s1 = 4
        s2 = site - s1
        
        rand_rho = random_density_matrix(2^site);

        c12 = apply_channel(apply_channel(rand_rho,s1,s2;act='A'), s2,s1;act='B');
        c21 = apply_channel(apply_channel(rand_rho,s2,s1;act='B'), s1,s2;act='A');
        println(norm(c12 - c21))
    end
end

9.884095077286555e-18
9.82654483774927e-18
8.502544908293946e-18


In [49]:
rand_rho = reshape(1:4^site, 2^site, 2^site)'*1.0
c12 = apply_channel(apply_channel(rand_rho,s1,s2;act='A'), s2,s1;act='B');
c21 = apply_channel(apply_channel(rand_rho,s2,s1;act='B'), s1,s2;act='A');

In [96]:
c12;